In [95]:
import pandas as pd

# Brute data from different sources:
# https://ai-project-inflation.s3.amazonaws.com/

brent_crude_prices = pd.read_excel("./Brent_crude_prices.xls", sheet_name=1, header=2)
brent_crude_prices.head()

,Date,Europe Brent Spot Price FOB (Dollars per Barrel)
0,1987-05-15,18.58
1,1987-06-15,18.86
2,1987-07-15,19.86
3,1987-08-15,18.98
4,1987-09-15,18.31


In [96]:

!pip install openpyxl
prime_rate = pd.read_excel("./Prime_Rate_History-WOWA.xlsx")
prime_rate.head()

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


,Date,Prime Rate,Bank of Canada Overnight Rate
0,January 1935,5.50,2.5
1,January 1936,5.21,2.5
2,January 1937,5.00,2.5
3,January 1938,5.00,2.5
4,January 1939,5.00,2.5


In [97]:
bond_rate = pd.read_csv("./StlouisfedCAD10yearbondrate.csv")
bond_rate.head()

,DATE,IRLTLT01CAM156N
0,2000-01-01,6.491500
1,2000-02-01,6.338571
2,2000-03-01,6.028261
3,2000-04-01,5.924211
4,2000-05-01,6.215000


In [98]:
fx_rates = pd.read_csv("./FX_RATES_MONTHLY_CSV.csv", )
fx_rates.head()

,Date,1 USD
0,1990-01-31,1.172038
1,1990-02-28,1.196479
2,1990-03-31,1.179986
3,1990-04-30,1.164095
4,1990-05-31,1.174700


In [99]:
manufacturing_rate = pd.read_csv("./Stlouisfeb_hourmanurate.csv")
manufacturing_rate.head()

,DATE,LCEAMN01CAM189S
0,2000-01-01,18.25
1,2000-02-01,18.33
2,2000-03-01,18.23
3,2000-04-01,18.27
4,2000-05-01,18.42


In [100]:
money_supply = pd.read_csv("./Stlousisfed_Canadain_Money_supply.csv")
money_supply.head()

,DATE,MANMM101CAM189S
0,2000-01-01,2.166880e+11
1,2000-02-01,2.216730e+11
2,2000-03-01,2.250820e+11
3,2000-04-01,2.284850e+11
4,2000-05-01,2.281010e+11


In [101]:
fed_fx_rates = pd.read_csv("./stlousisfedFXRates.csv")
fed_fx_rates.head()

,DATE,CCUSMA02CAM618N
0,2000-01-01,1.448226
1,2000-02-01,1.450341
2,2000-03-01,1.460354
3,2000-04-01,1.468305
4,2000-05-01,1.494781


In [102]:
cpi_inflation = pd.read_csv("./1810025601-CPI_and_Inflation_ydata.csv", header=8, nrows=9, skiprows={10,14})
cpi_inflation_inverted = pd.DataFrame.transpose(cpi_inflation, copy=False)
cpi_inflation_inverted.head()

,0,1,2,3,4,5,6,7,8
Alternative measures,Measure of core inflation based on a factor mo...,Measure of core inflation based on a weighted ...,Measure of core inflation based on a trimmed m...,"Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin...","Consumer Price Index (CPI), all-items excludin..."
May 2000,1.6,1.4,1.6,95.5,95.5,95.4,95.4,95.4,95.3
June 2000,1.7,1.6,1.8,95.8,95.8,96.0,95.6,95.7,95.8
July 2000,1.8,1.6,1.8,95.8,95.9,96.3,95.7,95.7,96.1
August 2000,1.9,1.7,1.9,96.0,96.0,96.2,95.9,95.9,96.1


In [103]:
tsx_data = pd.read_csv("./TSX_data_for_20_years.csv")
tsx_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2000-01-01,8271.700195,8755.599609,8018.600098,8481.099609,8481.099609,1.401281e+09
1,2000-02-01,8496.299805,9557.599609,8438.400391,9129.000000,9129.000000,9.071775e+08
2,2000-03-01,9317.900391,10176.700195,9191.900391,9462.400391,9462.400391,1.364784e+09
3,2000-04-01,9520.700195,9640.099609,8421.599609,9347.599609,9347.599609,1.839722e+09
4,2000-05-01,9399.099609,9620.500000,8807.400391,9252.000000,9252.000000,2.137767e+09


Merging data from the different files:

In [112]:
dataMerged = [brent_crude_prices,prime_rate,bond_rate, fx_rates, manufacturing_rate, money_supply, fed_fx_rates, cpi_inflation_inverted, tsx_data]

for x in dataMerged:
  print(x.shape)
  print(x.head())


(426, 2)
        Date  Europe Brent Spot Price FOB (Dollars per Barrel)
0 1987-05-15                                             18.58
1 1987-06-15                                             18.86
2 1987-07-15                                             19.86
3 1987-08-15                                             18.98
4 1987-09-15                                             18.31
(780, 3)
           Date  Prime Rate  Bank of Canada Overnight Rate
0  January 1935        5.50                            2.5
1  January 1936        5.21                            2.5
2  January 1937        5.00                            2.5
3  January 1938        5.00                            2.5
4  January 1939        5.00                            2.5
(272, 2)
         DATE  IRLTLT01CAM156N
0  2000-01-01         6.491500
1  2000-02-01         6.338571
2  2000-03-01         6.028261
3  2000-04-01         5.924211
4  2000-05-01         6.215000
(395, 2)
         Date     1 USD
0  1990-01-31  1.17203